In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import os
import time
import fastf1

# Optional: automated profiling
from ydata_profiling import ProfileReport

# Optional: pretty time axes / plots
from matplotlib.ticker import MaxNLocator
import calendar
from fastf1 import plotting


In [53]:
fastf1.Cache.clear_cache('../cache')
fastf1.Cache.enable_cache('../cache')

In [54]:
def load_all_races(data_dir="../data/processed"):
    all_files = [f for f in os.listdir(data_dir) if f.endswith(".csv")]
    dfs = []

    for file in all_files:
        df = pd.read_csv(os.path.join(data_dir, file))
        dfs.append(df)

    return pd.concat(dfs, ignore_index=True)

df = load_all_races()
print(f"✅ Loaded {len(df)} laps from {df['RaceName'].nunique()} races")
df.head(50)

✅ Loaded 77799 laps from 25 races


,Driver,Team,LapNumber,LapTime,Compound,StintNum,PitInTime,PitOutTime,TrackStatus,Position,Time,AirTemp,Humidity,Pressure,Rainfall,TrackTemp,WindSpeed,WindDirection,Year,RaceName
0,NOR,McLaren,1.0,100.424,MEDIUM,0,NaN,NaN,1,1.0,0 days 00:58:41.675000,30.8,75.0,1007.7,False,37.3,0.4,0,2024,Singapore_Grand_Prix
1,VER,Red Bull Racing,1.0,101.413,MEDIUM,1,NaN,NaN,1,2.0,0 days 00:58:42.664000,30.8,75.0,1007.7,False,37.3,0.4,0,2024,Singapore_Grand_Prix
2,HAM,Mercedes,1.0,102.702,SOFT,5,NaN,NaN,1,3.0,0 days 00:58:43.953000,30.8,75.0,1007.7,False,37.3,0.4,0,2024,Singapore_Grand_Prix
3,RUS,Mercedes,1.0,103.822,MEDIUM,3,NaN,NaN,1,4.0,0 days 00:58:45.073000,30.8,75.0,1007.7,False,37.3,0.4,0,2024,Singapore_Grand_Prix
4,PIA,McLaren,1.0,105.268,MEDIUM,2,NaN,NaN,1,5.0,0 days 00:58:46.519000,30.8,75.0,1007.7,False,37.3,0.4,0,2024,Singapore_Grand_Prix
5,HUL,Haas F1 Team,1.0,105.723,MEDIUM,8,NaN,NaN,1,6.0,0 days 00:58:46.974000,30.8,75.0,1007.7,False,37.3,0.4,0,2024,Singapore_Grand_Prix
6,ALO,Aston Martin,1.0,106.384,MEDIUM,7,NaN,NaN,1,7.0,0 days 00:58:47.635000,30.8,75.0,1007.7,False,37.3,0.4,0,2024,Singapore_Grand_Prix
7,LEC,Ferrari,1.0,107.090,MEDIUM,4,NaN,NaN,1,8.0,0 days 00:58:48.341000,30.8,75.0,1007.7,False,37.3,0.4,0,2024,Singapore_Grand_Prix
8,COL,Williams,1.0,107.696,MEDIUM,10,NaN,NaN,1,9.0,0 days 00:58:48.947000,30.8,75.0,1007.7,False,37.3,0.4,0,2024,Singapore_Grand_Prix
9,PER,Red Bull Racing,1.0,108.150,MEDIUM,9,NaN,NaN,1,10.0,0 days 00:58:49.401000,30.8,75.0,1007.7,False,37.3,0.4,0,2024,Singapore_Grand_Prix


In [55]:
cat_cols = ['Driver', 'Team', 'Compound', 'RaceName']
for col in cat_cols:
    df[col] = df[col].astype('category')

In [56]:
print(df.dtypes)

Driver           category
Team             category
LapNumber         float64
LapTime           float64
Compound         category
StintNum            int64
PitInTime          object
PitOutTime         object
TrackStatus         int64
Position          float64
Time               object
AirTemp           float64
Humidity          float64
Pressure          float64
Rainfall             bool
TrackTemp         float64
WindSpeed         float64
WindDirection       int64
Year                int64
RaceName         category
dtype: object


In [57]:
df.head(100)

,Driver,Team,LapNumber,LapTime,Compound,StintNum,PitInTime,PitOutTime,TrackStatus,Position,Time,AirTemp,Humidity,Pressure,Rainfall,TrackTemp,WindSpeed,WindDirection,Year,RaceName
0,NOR,McLaren,1.0,100.424,MEDIUM,0,NaN,NaN,1,1.0,0 days 00:58:41.675000,30.8,75.0,1007.7,False,37.3,0.4,0,2024,Singapore_Grand_Prix
1,VER,Red Bull Racing,1.0,101.413,MEDIUM,1,NaN,NaN,1,2.0,0 days 00:58:42.664000,30.8,75.0,1007.7,False,37.3,0.4,0,2024,Singapore_Grand_Prix
2,HAM,Mercedes,1.0,102.702,SOFT,5,NaN,NaN,1,3.0,0 days 00:58:43.953000,30.8,75.0,1007.7,False,37.3,0.4,0,2024,Singapore_Grand_Prix
3,RUS,Mercedes,1.0,103.822,MEDIUM,3,NaN,NaN,1,4.0,0 days 00:58:45.073000,30.8,75.0,1007.7,False,37.3,0.4,0,2024,Singapore_Grand_Prix
4,PIA,McLaren,1.0,105.268,MEDIUM,2,NaN,NaN,1,5.0,0 days 00:58:46.519000,30.8,75.0,1007.7,False,37.3,0.4,0,2024,Singapore_Grand_Prix
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,RIC,RB,5.0,99.585,SOFT,17,NaN,NaN,1,16.0,0 days 01:05:32.704000,31.1,74.0,1007.8,False,36.6,1.4,195,2024,Singapore_Grand_Prix
96,GAS,Alpine,5.0,99.582,MEDIUM,16,NaN,NaN,1,17.0,0 days 01:05:33.635000,31.1,74.0,1007.8,False,36.6,1.4,195,2024,Singapore_Grand_Prix
97,STR,Aston Martin,5.0,99.613,HARD,13,NaN,NaN,1,18.0,0 days 01:05:34.600000,31.1,74.0,1007.8,False,36.6,1.4,195,2024,Singapore_Grand_Prix
98,BOT,Kick Sauber,5.0,99.848,HARD,15,NaN,NaN,1,19.0,0 days 01:05:36.402000,31.1,74.0,1007.8,False,36.6,1.4,195,2024,Singapore_Grand_Prix


In [58]:
df['RaceID'] = df['Year'].astype(str) + '_' + df['RaceName'].astype(str)

In [59]:
fastf1.Cache.enable_cache('../cache')  # adjust path if needed

def remove_dnf_laps(df, races_to_patch):
    patched_dfs = []

    for race_id in races_to_patch:
        print(f"\n🔍 Trying {race_id}")
        year = int(race_id.split("_")[0])
        event_name = "_".join(race_id.split("_")[1:]).replace("_", " ")

        try:
            session = fastf1.get_session(year, event_name, 'R')
            session.load()

            results = session.results[['Abbreviation', 'Position', 'Status']].copy()
            results.rename(columns={
                'Abbreviation': 'Driver',
                'Position': 'FinalPosition',
                'Status': 'FinalStatus'
            }, inplace=True)
            results['Driver'] = results['Driver'].astype(str)

            df_race = df[df['RaceID'] == race_id].copy()
            df_race['Driver'] = df_race['Driver'].astype(str)

            df_race = df_race.merge(results, on='Driver', how='left')

            if df_race['FinalStatus'].isna().all():
                print(f"⚠️ No driver matches in {race_id}, skipping.")
                continue

            df_race = df_race[df_race['FinalStatus'].str.contains("Finished|Lap", case=False, na=False)]
            patched_dfs.append(df_race)
            print(f"✅ Patched: {race_id} ({len(df_race)} rows)")

        except Exception as e:
            print(f"❌ Failed for {race_id}: {e}")
            continue

    if not patched_dfs:
        raise ValueError("❌ No races were patched. Check cache/network or inputs.")

    return pd.concat(patched_dfs, ignore_index=True)


In [60]:
races = df['RaceID'].unique().tolist()
df = remove_dnf_laps(df, races)

core           INFO 	Loading data for Singapore Grand Prix - Race [v3.5.3]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...



🔍 Trying 2024_Singapore_Grand_Prix


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!

✅ Patched: 2024_Singapore_Grand_Prix (1161 rows)

🔍 Trying 2024_Qatar_Grand_Prix


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!

...

✅ Patched: 2023_Dutch_Grand_Prix (1193 rows)

🔍 Trying 2022_Japanese_Grand_Prix


logger      WARNING 	Failed to load session info data!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
core        WARNING 	Failed to load extended driver information!
Request for URL https://api.jolpi.ca/ergast/f1/2022/18/results.json failed; using cached response
Traceback (most recent call last):
  File "/opt/miniconda3/envs/cs7641/lib/python3.10/site-packages/requests_cache/session.py", line 286, in _resend
    response = self._send_and_cache(request, actions, cached_response, **kwargs)
  File "/opt/miniconda3/envs/cs7641/lib/python3.10/site-packages/requests_cache/session.py", line 254, in _send_and_cache
    response = super().send(request, **kwargs)
  File "/opt/miniconda3/envs/cs7641/lib/python3.10/site-packages/fastf1/req.py", line 134, in send
    lim.limit()
  File "/opt/miniconda3/envs/cs7641/lib/python3.10/site-packages/fastf1/req.py", line 104, in limit
    raise RateLimitExceededError(self._info)
fas

✅ Patched: 2022_Japanese_Grand_Prix (432 rows)

🔍 Trying 2023_Saudi_Arabian_Grand_Prix


logger      WARNING 	Failed to load session info data!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
core        WARNING 	Failed to load extended driver information!
Request for URL https://api.jolpi.ca/ergast/f1/2023/2/results.json failed; using cached response
Traceback (most recent call last):
  File "/opt/miniconda3/envs/cs7641/lib/python3.10/site-packages/requests_cache/session.py", line 286, in _resend
    response = self._send_and_cache(request, actions, cached_response, **kwargs)
  File "/opt/miniconda3/envs/cs7641/lib/python3.10/site-packages/requests_cache/session.py", line 254, in _send_and_cache
    response = super().send(request, **kwargs)
  File "/opt/miniconda3/envs/cs7641/lib/python3.10/site-packages/fastf1/req.py", line 134, in send
    lim.limit()
  File "/opt/miniconda3/envs/cs7641/lib/python3.10/site-packages/fastf1/req.py", line 104, in limit
    raise RateLimitExceededError(self._info)
fast

✅ Patched: 2023_Saudi_Arabian_Grand_Prix (883 rows)


In [47]:

df.head(10)

,Driver,Team,LapNumber,LapTime,Compound,StintNum,PitInTime,PitOutTime,TrackStatus,Position,...,Humidity,Pressure,Rainfall,TrackTemp,WindSpeed,WindDirection,Year,RaceName,RaceID,LapStint
0,NOR,McLaren,1.0,100.424,MEDIUM,0,NaN,NaN,1,1.0,...,75.0,1007.7,False,37.3,0.4,0,2024,Singapore_Grand_Prix,2024_Singapore_Grand_Prix,1
1,VER,Red Bull Racing,1.0,101.413,MEDIUM,1,NaN,NaN,1,2.0,...,75.0,1007.7,False,37.3,0.4,0,2024,Singapore_Grand_Prix,2024_Singapore_Grand_Prix,1
2,HAM,Mercedes,1.0,102.702,SOFT,5,NaN,NaN,1,3.0,...,75.0,1007.7,False,37.3,0.4,0,2024,Singapore_Grand_Prix,2024_Singapore_Grand_Prix,1
3,RUS,Mercedes,1.0,103.822,MEDIUM,3,NaN,NaN,1,4.0,...,75.0,1007.7,False,37.3,0.4,0,2024,Singapore_Grand_Prix,2024_Singapore_Grand_Prix,1
4,PIA,McLaren,1.0,105.268,MEDIUM,2,NaN,NaN,1,5.0,...,75.0,1007.7,False,37.3,0.4,0,2024,Singapore_Grand_Prix,2024_Singapore_Grand_Prix,1
5,HUL,Haas F1 Team,1.0,105.723,MEDIUM,8,NaN,NaN,1,6.0,...,75.0,1007.7,False,37.3,0.4,0,2024,Singapore_Grand_Prix,2024_Singapore_Grand_Prix,1
6,ALO,Aston Martin,1.0,106.384,MEDIUM,7,NaN,NaN,1,7.0,...,75.0,1007.7,False,37.3,0.4,0,2024,Singapore_Grand_Prix,2024_Singapore_Grand_Prix,1
7,LEC,Ferrari,1.0,107.090,MEDIUM,4,NaN,NaN,1,8.0,...,75.0,1007.7,False,37.3,0.4,0,2024,Singapore_Grand_Prix,2024_Singapore_Grand_Prix,1
8,COL,Williams,1.0,107.696,MEDIUM,10,NaN,NaN,1,9.0,...,75.0,1007.7,False,37.3,0.4,0,2024,Singapore_Grand_Prix,2024_Singapore_Grand_Prix,1
9,PER,Red Bull Racing,1.0,108.150,MEDIUM,9,NaN,NaN,1,10.0,...,75.0,1007.7,False,37.3,0.4,0,2024,Singapore_Grand_Prix,2024_Singapore_Grand_Prix,1


In [43]:
df['LapStint'] = df.groupby(['RaceID', 'Driver', 'StintNum']).cumcount() + 1

/var/folders/1r/y7d2wcxs0h12k2jxr_xw_h200000gn/T/ipykernel_2696/1431933618.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['LapStint'] = df.groupby(['RaceID', 'Driver', 'StintNum']).cumcount() + 1


In [ ]:
df = remove_dnf_laps(df)

In [45]:
df.isna().sum().sort_values(ascending=False)


PitOutTime       75448
PitInTime        75263
Driver               0
Humidity             0
RaceID               0
RaceName             0
Year                 0
WindDirection        0
WindSpeed            0
TrackTemp            0
Rainfall             0
Pressure             0
AirTemp              0
Team                 0
Time                 0
Position             0
TrackStatus          0
StintNum             0
Compound             0
LapTime              0
LapNumber            0
LapStint             0
dtype: int64

In [15]:
df.drop(columns=['PitInTime', 'PitOutTime'], inplace=True)

In [46]:
print(df.dtypes)

Driver           category
Team             category
LapNumber         float64
LapTime           float64
Compound         category
StintNum            int64
PitInTime          object
PitOutTime         object
TrackStatus         int64
Position          float64
Time               object
AirTemp           float64
Humidity          float64
Pressure          float64
Rainfall             bool
TrackTemp         float64
WindSpeed         float64
WindDirection       int64
Year                int64
RaceName         category
RaceID             object
LapStint            int64
dtype: object


In [20]:
os.makedirs('data', exist_ok=True)
df.to_csv('data/cleaned_f1_data.csv', index = False)